Download and Clean Dataset


In [1]:
import pandas as pd
import numpy as np

The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:

1. Cement

2. Blast Furnace Slag

3. Fly Ash

4. Water

5. Superplasticizer

6. Coarse Aggregate

7. Fine Aggregate

In [2]:
concrete_data = pd.read_csv('concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


the first row sample has 540 cubic meter of cement, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa.

In [3]:
concrete_data.shape

(1030, 9)

The sample has 1030 rows and 9 columns. Because of the few samples, we have to be careful not to overfit the training data.  

Do we have missing values?

In [4]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [5]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.

*Split data into predictors and target*
The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns

In [6]:
concrete_data_columns = concrete_data.columns

concrete_data_columns


Index(['Cement', 'Blast Furnace Slag', 'Fly Ash', 'Water', 'Superplasticizer',
       'Coarse Aggregate', 'Fine Aggregate', 'Age', 'Strength'],
      dtype='object')

In [7]:
predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

In [8]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [9]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

In [10]:
n_cols = predictors.shape[1] # number of predictors

Lets normalize de data because there are different features that have different ranges

In [11]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


Import keras

In [12]:
import keras

Using TensorFlow backend.
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyterlab/con

In [13]:
from keras.models import Sequential
from keras.layers import Dense

Build a Neural Network

Lets create a model with 3 hidden layers of 10 neurons and ReLU activation function

In [14]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

 Randomly split the data into a training and test sets by holding 30% of the data for testing. Let's  can use the train_test_split helper function from Scikit-learn.

In [15]:

from sklearn.model_selection import train_test_split

In [16]:

X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=20)

Lets train and test the model

In [17]:


model = regression_model()

Lets train the model using 100 epoch

In [18]:
epochs = 50
model.fit(X_train, y_train, epochs=epochs, verbose=1)

Epoch 1/50
721/721 [==============================] - 1s 1ms/step - loss: 1559.0431
Epoch 2/50
721/721 [==============================] - 0s 416us/step - loss: 1533.5241
Epoch 3/50
721/721 [==============================] - 0s 414us/step - loss: 1504.4447
Epoch 4/50
721/721 [==============================] - 0s 443us/step - loss: 1463.9711
Epoch 5/50
721/721 [==============================] - 0s 441us/step - loss: 1401.0329
Epoch 6/50
721/721 [==============================] - 0s 367us/step - loss: 1302.9851
Epoch 7/50
721/721 [==============================] - 0s 407us/step - loss: 1152.5783
Epoch 8/50
721/721 [==============================] - 0s 390us/step - loss: 940.7585
Epoch 9/50
721/721 [==============================] - 0s 389us/step - loss: 687.7979
Epoch 10/50
721/721 [==============================] - 0s 418us/step - loss: 452.7936
Epoch 11/50
721/721 [==============================] - 0s 468us/step - loss: 300.5459
Epoch 12/50
721/721 [==============================] - 0s 

 Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength. You can use the mean_squared_error function from Scikit-learn.

In [19]:
loss_val = model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
loss_val

309/309 [==============================] - 0s 373us/step


88.8647267610124

In [20]:
from sklearn.metrics import mean_squared_error

In [21]:

mean_square_error = mean_squared_error(y_test, y_pred)
mean = np.mean(mean_square_error)
standard_deviation = np.std(mean_square_error)
print(mean, standard_deviation)

88.864724160334 0.0


create a list of 50 mean squared errors.

In [22]:
mean_square_error_list = []
for i in range (0, 50):
    
    X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=20)

    model.fit(X_train, y_train, epochs=50, verbose=0)
    loss_val = model.evaluate(X_test, y_test, verbose=0)
    print("Iteration: " +str(i) +" Loss value: "+ str(loss_val))
    y_pred = model.predict(X_test)
    mean_square_error = mean_squared_error(y_test, y_pred)
    mean = np.mean(mean_square_error)
    mean_square_error_list.append(mean)
    


Iteration: 0 Loss value: 43.23910213828473
Iteration: 1 Loss value: 39.314332437360946
Iteration: 2 Loss value: 38.26408234383296
Iteration: 3 Loss value: 38.00452971226961
Iteration: 4 Loss value: 38.7267925916752
Iteration: 5 Loss value: 40.104715217664406
Iteration: 6 Loss value: 39.64050349757124
Iteration: 7 Loss value: 41.25342386980273
Iteration: 8 Loss value: 41.066437446569545
Iteration: 9 Loss value: 40.94763308281266
Iteration: 10 Loss value: 41.53829422737788
Iteration: 11 Loss value: 40.85279442500142
Iteration: 12 Loss value: 40.49491511965261
Iteration: 13 Loss value: 40.49702414108326
Iteration: 14 Loss value: 40.19642299664445
Iteration: 15 Loss value: 40.182556004200165
Iteration: 16 Loss value: 39.90782658104758
Iteration: 17 Loss value: 40.19387554427952
Iteration: 18 Loss value: 40.308108505693454
Iteration: 19 Loss value: 40.46993045745158
Iteration: 20 Loss value: 40.201782806791535
Iteration: 21 Loss value: 40.915604057435466
Iteration: 22 Loss value: 40.3170912

In [23]:
mean_squared_errors = np.array(mean_square_error_list)
mean = np.mean(mean_squared_errors)
standard_deviation = np.std(mean_squared_errors)

print(f"Mean {mean} and standard deviation {standard_deviation} ")

Mean 40.754228733515184 and standard deviation 0.9315320331512423 


The inclusion of 3 layers in the model improves the model precission